In [350]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import svm

from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.multioutput import MultiOutputRegressor

%matplotlib inline

In [329]:
data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")

bts_data = pd.read_csv("Dados/dados_BTSs.csv")

In [330]:
data.drop_duplicates(inplace=True)
data.shape
data = data.drop(['pontoId'], axis=1)
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())
data.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [332]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [333]:
x = data.drop(['lat', 'lon'], axis=1)
x.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [334]:
y = data[['lat', 'lon']]
y.describe()

,lat,lon
count,1500.000000,1500.000000
mean,-8.070097,-34.894814
std,0.004232,0.004354
min,-8.077570,-34.906830
25%,-8.073197,-34.897387
50%,-8.069940,-34.895346
75%,-8.067113,-34.891169
max,-8.059339,-34.885067


In [352]:
transformerY = RobustScaler().fit(y)
y_scaled = transformerY.transform(y)

transformerX = RobustScaler().fit(x)
x_scaled = transformerX.transform(x)

X_train, X_test, y_train, y_test = train_test_split(x, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1050, 12) (450, 12) (1050, 2) (450, 2)


In [353]:
multioutputregressor = MultiOutputRegressor(model_xgb).fit(X_train, y_train)

In [354]:
print (transformerY.inverse_transform(multioutputregressor.predict(X_test)))

[[ -8.072628  -34.89193  ]
 [ -8.070054  -34.89671  ]
 [ -8.072618  -34.89438  ]
 [ -8.072625  -34.89221  ]
 [ -8.072662  -34.895714 ]
 [ -8.076581  -34.897648 ]
 [ -8.070217  -34.896957 ]
 [ -8.061157  -34.90531  ]
 [ -8.0746765 -34.890553 ]
 [ -8.074773  -34.891575 ]
 [ -8.070898  -34.899998 ]
 [ -8.068502  -34.891712 ]
 [ -8.073215  -34.896297 ]
 [ -8.0695    -34.89061  ]
 [ -8.070901  -34.90083  ]
 [ -8.07478   -34.89201  ]
 [ -8.068807  -34.888813 ]
 [ -8.065305  -34.890976 ]
 [ -8.06563   -34.886036 ]
 [ -8.072947  -34.89738  ]
 [ -8.072651  -34.896164 ]
 [ -8.069087  -34.89699  ]
 [ -8.070308  -34.89598  ]
 [ -8.076672  -34.899998 ]
 [ -8.076142  -34.89985  ]
 [ -8.071605  -34.900898 ]
 [ -8.071584  -34.891804 ]
 [ -8.06846   -34.89115  ]
 [ -8.064142  -34.88751  ]
 [ -8.065432  -34.88759  ]
 [ -8.072881  -34.894276 ]
 [ -8.070164  -34.8967   ]
 [ -8.062762  -34.886417 ]
 [ -8.070422  -34.89489  ]
 [ -8.065878  -34.8888   ]
 [ -8.073416  -34.896088 ]
 [ -8.0710945 -34.90083  ]
 

In [355]:
print (np.mean((transformerY.inverse_transform(multioutputregressor.predict(X_test)) - transformerY.inverse_transform(y_test))**2, axis=0))

[2.77745908e-07 4.05578620e-07]


In [356]:
from math import sin, cos, sqrt, atan2, radians


def calculateDistance(lat1, lon1, lat2, lon2):
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [357]:
v = transformerY.inverse_transform(multioutputregressor.predict(X_test))

lat1 = v[:,0]
lon1 = v[:,1]
lat2 = transformerY.inverse_transform(y_test)[:,0]
lon2 = transformerY.inverse_transform(y_test)[:,1]

dist = calculateDistance(lat1, lon2, lat2, lon2)

print(dist.mean())

2.5987551878874


In [358]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train,y_train)

out2 = transformerY.inverse_transform(regr.predict(X_test))

lat3 = out2[:,0]
lon3 = out2[:,1]
lat4 = transformerY.inverse_transform(y_test)[:,0]
lon4 = transformerY.inverse_transform(y_test)[:,1]

dist2 = calculateDistance(lat3, lon3, lat4, lon4)
print(dist2.mean())

20.6901071059817
